In [48]:
import os
import tabula
import pandas as pd
import re
from PyPDF2 import PdfReader

    

In [5]:
file = '../Downloaded Data/2021/Abu Dhabi/2021_22_uae_f1_p1_timing_firstpracticesessionlaptimes_v01.pdf'

In [226]:
reader = PdfReader(file)
text = ""
for page in reader.pages:
    text += page.extract_text() + "\n"
    
text = text.replace("\n","")
text = re.sub('(\.\d{3})(\d+)',r'\1 \2', text)
text = re.sub('(\d+)(\s{2,})',r'-\1 ', text)
text = re.split('-', text)
text = [x for x in text if x]
times = pd.DataFrame()
for i in text:
    i = re.sub("(\.\d{3})[a-zA-Z].+",r"\1",i)
    i = re.sub("(\d)(P)(\d)",r"\1 \2 \3", i)
    i = re.sub("NOTIME", " ", i)
    i = re.sub(" (\d{1,2} (P )?\d:\d{2}\.\d{3})",r"-\1", i)
    i = re.split("-", i)
    if len(i) > 1:
        driver_info = re.split(' ', i[0], maxsplit = 1)
        driver_name = driver_info[1]
        driver_number = driver_info[0]
        driver_laptimes = i[1:]
        for lap in driver_laptimes:
            lap_pit = "P" if "P" in lap else ""
            lap = re.sub("P ","", lap)
            lap_info = re.split(' ', lap, maxsplit = 1)
            lap_number = int(lap_info[0])
            lap_time = lap_info[1]
            temp_df = pd.DataFrame({"Driver Name":[driver_name], "Driver Number":[driver_number], "Lap Number":[lap_number], "Lap Time":[lap_time], "Pit":[lap_pit]})
            times = times.append(temp_df)

,Driver Name,Driver Number,Lap Number,Lap Time,Pit
0,Daniel RICCIARDO,3,16,1:31.187,
0,Daniel RICCIARDO,3,17,1:30.502,
0,Daniel RICCIARDO,3,18,1:30.892,
0,Daniel RICCIARDO,3,19,1:32.179,
0,Daniel RICCIARDO,3,20,1:31.220,
...,...,...,...,...,...
0,Antonio GIOVINAZZI,99,21,1:30.946,
0,Antonio GIOVINAZZI,99,22,1:34.338,
0,Antonio GIOVINAZZI,99,23,1:30.697,
0,Antonio GIOVINAZZI,99,24,1:58.919,


In [117]:
[1,2]+[3,4]

[1, 2, 3, 4]